# 📚 MuJoCo入门教程 (Part 1) - 基础篇
# MuJoCo Tutorial Part 1 - Basics

---

## 🎯 学习目标

通过本教程，你将学会：

1. ✅ MuJoCo的基本概念和架构
2. ✅ 如何编写XML配置文件
3. ✅ 创建基本的3D几何形状
4. ✅ 添加材质和颜色
5. ✅ 设置物理属性（质量、摩擦力等）
6. ✅ 添加光源和摄像头
7. ✅ 可视化和交互控制

---

## 📖 目录

1. [环境设置](#1-环境设置)
2. [MuJoCo核心概念](#2-mujoco核心概念)
3. [第一个XML文件](#3-第一个xml文件)
4. [添加几何形状](#4-添加几何形状)
5. [材质和颜色](#5-材质和颜色)
6. [物理属性](#6-物理属性)
7. [摄像头和光源](#7-摄像头和光源)
8. [可视化和渲染](#8-可视化和渲染)
9. [保存自定义场景](#9-保存自定义场景)

---

## 1. 环境设置
## 1. Environment Setup

首先，让我们导入必要的库并设置环境变量。

In [ ]:
# 📦 导入必要的库
# Import necessary libraries

import os
import numpy as np
import mujoco
import matplotlib.pyplot as plt
from IPython.display import Image, display
import xml.etree.ElementTree as ET
from xml.dom import minidom

print("✅ 库导入成功！")
print(f"   MuJoCo版本: {mujoco.__version__}")
print(f"   NumPy版本: {np.__version__}")

In [ ]:
# 🎨 设置环境变量（GPU渲染）
# Set environment variables for GPU rendering

os.environ['MUJOCO_GL'] = 'egl'  # 使用EGL后端进行GPU加速
os.environ['DISPLAY'] = ':0'

print("✅ 环境变量设置完成！")
print(f"   渲染后端: {os.environ['MUJOCO_GL']}")

In [ ]:
# 📁 创建教程工作目录
# Create tutorial working directory

tutorial_dir = './tutorial_scenes'
os.makedirs(tutorial_dir, exist_ok=True)

print(f"✅ 教程目录创建成功: {tutorial_dir}")
print(f"   所有自定义场景将保存在这个目录中")

---

## 2. MuJoCo核心概念
## 2. MuJoCo Core Concepts

### 🏗️ MuJoCo架构

MuJoCo (Multi-Joint dynamics with Contact) 是一个物理仿真引擎，主要由以下部分组成：

```
┌─────────────────────────────────────────┐
│         XML配置文件 (Scene File)         │
│  定义整个仿真世界的所有元素和属性          │
└──────────────┬──────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│         MjModel (模型对象)               │
│  包含所有静态信息（几何、质量、约束等）    │
└──────────────┬──────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│         MjData (数据对象)                │
│  包含所有动态信息（位置、速度、力等）      │
└──────────────┬──────────────────────────┘
               │
               ▼
┌─────────────────────────────────────────┐
│      mj_step() 物理仿真步进函数          │
│  根据物理定律更新系统状态                 │
└─────────────────────────────────────────┘
```

### 📝 XML文件结构

MuJoCo使用XML格式定义仿真场景。基本结构如下：

```xml
<mujoco model="场景名称">
    <option>        <!-- 仿真选项 -->
    <asset>         <!-- 资源（网格、材质、纹理） -->
    <worldbody>     <!-- 世界物体层级结构 -->
        <body>      <!-- 刚体 -->
            <geom>  <!-- 几何形状 -->
            <joint> <!-- 关节 -->
        </body>
    </worldbody>
    <actuator>      <!-- 执行器（电机） -->
</mujoco>
```

### 🔑 关键术语

- **geom**: 几何体，定义物体的形状和外观
- **body**: 刚体，可以包含多个geom和joint
- **joint**: 关节，连接两个body，允许相对运动
- **actuator**: 执行器，对joint施加力或力矩
- **site**: 位置标记，用于传感器和可视化
- **camera**: 摄像头，用于渲染和观察

---

## 3. 第一个XML文件
## 3. Your First XML File

让我们从最简单的MuJoCo场景开始：一个包含地面和一个球的场景。

In [ ]:
# 🎈 示例1: 最简单的MuJoCo场景
# Example 1: The simplest MuJoCo scene

xml_content_simple = """
<mujoco model="simple_scene">
    <!-- 这是最简单的MuJoCo场景 -->
    <!-- This is the simplest MuJoCo scene -->
    
    <worldbody>
        <!-- 地面：一个大的平面 -->
        <!-- Ground: a large plane -->
        <geom name="ground" type="plane" size="2 2 0.1" rgba="0.8 0.8 0.8 1"/>
        
        <!-- 球体：悬浮在空中 -->
        <!-- Sphere: floating in the air -->
        <body name="ball" pos="0 0 1">
            <freejoint/>  <!-- 自由关节：允许6自由度运动 -->
            <geom name="ball_geom" type="sphere" size="0.1" rgba="1 0 0 1"/>
        </body>
    </worldbody>
</mujoco>
"""

# 保存XML文件
xml_path_simple = os.path.join(tutorial_dir, '01_simple_scene.xml')
with open(xml_path_simple, 'w') as f:
    f.write(xml_content_simple)

print(f"✅ XML文件创建成功: {xml_path_simple}")
print("\n📝 场景内容:")
print("   - 灰色地面 (plane)")
print("   - 红色球体 (sphere) 位于 (0, 0, 1)")
print("   - 球体具有自由关节，可以自由下落")

In [ ]:
# 🎬 加载并可视化场景
# Load and visualize the scene

# 使用绝对路径加载（重要！）
xml_path_abs = os.path.abspath(xml_path_simple)
model = mujoco.MjModel.from_xml_path(xml_path_abs)
data = mujoco.MjData(model)

print("✅ 模型加载成功！")
print(f"\n📊 模型信息:")
print(f"   物体数量: {model.nbody}")
print(f"   几何体数量: {model.ngeom}")
print(f"   关节数量: {model.njnt}")
print(f"   时间步长: {model.opt.timestep} 秒")

In [ ]:
# 🎥 渲染单帧图像
# Render a single frame

# 创建渲染器
renderer = mujoco.Renderer(model, height=480, width=640)

# 更新场景并渲染
renderer.update_scene(data)
image = renderer.render()

# 显示图像
plt.figure(figsize=(10, 7))
plt.imshow(image)
plt.title('第一个MuJoCo场景 - 初始状态', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ 图像渲染成功！")
print("   你应该看到一个红色球体悬浮在灰色地面上方")

In [ ]:
# ⏱️ 运行物理仿真
# Run physics simulation

print("🎬 开始仿真...球体将自由下落并弹跳")
print("\n时间步\t高度(m)\t垂直速度(m/s)")
print("-" * 45)

# 运行200步仿真
for i in range(200):
    mujoco.mj_step(model, data)
    
    # 每20步打印一次状态
    if i % 20 == 0:
        ball_pos = data.qpos[2]  # Z坐标（高度）
        ball_vel = data.qvel[2]  # Z方向速度
        print(f"{i}\t{ball_pos:.4f}\t\t{ball_vel:.4f}")

# 渲染最终状态
renderer.update_scene(data)
image_final = renderer.render()

# 显示对比图
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].imshow(image)
axes[0].set_title('初始状态 (t=0)', fontsize=12)
axes[0].axis('off')

axes[1].imshow(image_final)
axes[1].set_title('最终状态 (t=200 steps)', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ 仿真完成！球体已落到地面上")

---

## 4. 添加几何形状
## 4. Adding Geometric Shapes

MuJoCo支持多种基本几何形状：
- `plane` - 平面
- `sphere` - 球体
- `box` - 立方体
- `cylinder` - 圆柱体
- `capsule` - 胶囊体
- `ellipsoid` - 椭球体
- `mesh` - 自定义网格

让我们创建一个包含多种形状的场景。

In [ ]:
# 🎨 示例2: 多种几何形状
# Example 2: Multiple geometric shapes

xml_content_shapes = """
<mujoco model="shapes_gallery">
    <option gravity="0 0 -9.81"/>
    
    <worldbody>
        <!-- 地面 -->
        <geom name="ground" type="plane" size="3 3 0.1" rgba="0.9 0.9 0.9 1"/>
        
        <!-- 光源 -->
        <light pos="0 0 3" dir="0 0 -1" diffuse="1 1 1"/>
        
        <!-- 球体 (Sphere) -->
        <body name="sphere" pos="-1.5 0 0.5">
            <freejoint/>
            <geom type="sphere" size="0.15" rgba="1 0 0 1"/>
        </body>
        
        <!-- 立方体 (Box) -->
        <body name="box" pos="-0.5 0 0.5">
            <freejoint/>
            <geom type="box" size="0.12 0.12 0.12" rgba="0 1 0 1"/>
        </body>
        
        <!-- 圆柱体 (Cylinder) -->
        <body name="cylinder" pos="0.5 0 0.5">
            <freejoint/>
            <geom type="cylinder" size="0.1 0.2" rgba="0 0 1 1"/>
        </body>
        
        <!-- 胶囊体 (Capsule) -->
        <body name="capsule" pos="1.5 0 0.5">
            <freejoint/>
            <geom type="capsule" size="0.08 0.15" rgba="1 1 0 1"/>
        </body>
        
        <!-- 椭球体 (Ellipsoid) -->
        <body name="ellipsoid" pos="0 1.5 0.5">
            <freejoint/>
            <geom type="ellipsoid" size="0.2 0.1 0.08" rgba="1 0 1 1"/>
        </body>
    </worldbody>
</mujoco>
"""

# 保存XML文件
xml_path_shapes = os.path.join(tutorial_dir, '02_shapes_gallery.xml')
with open(xml_path_shapes, 'w') as f:
    f.write(xml_content_shapes)

print(f"✅ 形状画廊场景创建成功: {xml_path_shapes}")
print("\n🎨 包含的形状:")
print("   🔴 红色球体 (sphere)")
print("   🟢 绿色立方体 (box)")
print("   🔵 蓝色圆柱体 (cylinder)")
print("   🟡 黄色胶囊体 (capsule)")
print("   🟣 紫色椭球体 (ellipsoid)")

In [ ]:
# 📸 加载并渲染形状画廊
# Load and render the shapes gallery

xml_path_abs = os.path.abspath(xml_path_shapes)
model_shapes = mujoco.MjModel.from_xml_path(xml_path_abs)
data_shapes = mujoco.MjData(model_shapes)

# 创建渲染器
renderer_shapes = mujoco.Renderer(model_shapes, height=480, width=640)

# 渲染初始状态
renderer_shapes.update_scene(data_shapes)
image_shapes = renderer_shapes.render()

plt.figure(figsize=(12, 8))
plt.imshow(image_shapes)
plt.title('MuJoCo几何形状画廊', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ 形状画廊渲染成功！")
print(f"   物体总数: {model_shapes.nbody}")
print(f"   几何体总数: {model_shapes.ngeom}")

### 📏 几何形状的尺寸参数

不同类型的几何体使用不同的尺寸参数：

| 形状 | size参数 | 说明 |
|------|----------|------|
| plane | `size="x y z"` | x,y: 半边长, z: 未使用 |
| sphere | `size="r"` | r: 半径 |
| box | `size="x y z"` | x,y,z: 半边长 |
| cylinder | `size="r h"` | r: 半径, h: 半高 |
| capsule | `size="r h"` | r: 半径, h: 半高（不含半球） |
| ellipsoid | `size="x y z"` | x,y,z: 三个半轴长度 |

**注意**: 大多数尺寸参数使用的是**半长度**，而不是全长度！

In [ ]:
# 🎲 练习1: 创建你自己的形状
# Exercise 1: Create your own shape

# TODO: 修改以下参数，创建一个自定义的几何体
# 尝试改变: 形状类型、位置、大小、颜色

custom_shape_type = "box"  # 改成 "sphere", "cylinder", "capsule" 等
custom_position = "0 0 1"  # X Y Z 坐标
custom_size = "0.2 0.2 0.2"  # 根据形状类型调整
custom_color = "0.5 0.5 1 1"  # RGBA (0-1)

xml_custom = f"""
<mujoco model="my_custom_shape">
    <worldbody>
        <geom name="ground" type="plane" size="2 2 0.1" rgba="0.9 0.9 0.9 1"/>
        <light pos="0 0 3" dir="0 0 -1"/>
        
        <body name="my_object" pos="{custom_position}">
            <freejoint/>
            <geom type="{custom_shape_type}" size="{custom_size}" rgba="{custom_color}"/>
        </body>
    </worldbody>
</mujoco>
"""

# 保存并加载
xml_path_custom = os.path.join(tutorial_dir, '03_my_shape.xml')
with open(xml_path_custom, 'w') as f:
    f.write(xml_custom)

model_custom = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_custom))
data_custom = mujoco.MjData(model_custom)

# 渲染
renderer_custom = mujoco.Renderer(model_custom, height=480, width=640)
renderer_custom.update_scene(data_custom)
image_custom = renderer_custom.render()

plt.figure(figsize=(10, 7))
plt.imshow(image_custom)
plt.title(f'我的自定义形状: {custom_shape_type}', fontsize=14)
plt.axis('off')
plt.show()

print(f"✅ 自定义形状创建成功: {custom_shape_type}")

---

## 5. 材质和颜色
## 5. Materials and Colors

MuJoCo支持多种方式定义物体的外观：
1. 直接使用RGBA颜色
2. 使用材质（Material）
3. 使用纹理（Texture）

In [ ]:
# 🎨 示例3: 材质和颜色
# Example 3: Materials and colors

xml_content_materials = """
<mujoco model="materials_demo">
    <!-- 定义材质 -->
    <asset>
        <!-- 金属材质：高反光 -->
        <material name="metal" rgba="0.7 0.7 0.8 1" 
                  specular="1" shininess="0.9" reflectance="0.5"/>
        
        <!-- 塑料材质：低反光 -->
        <material name="plastic" rgba="1 0.5 0.2 1" 
                  specular="0.5" shininess="0.3" reflectance="0"/>
        
        <!-- 半透明材质 -->
        <material name="glass" rgba="0.5 0.8 1 0.5" 
                  specular="1" shininess="0.95" reflectance="0.3"/>
    </asset>
    
    <worldbody>
        <geom name="ground" type="plane" size="3 3 0.1" rgba="0.2 0.3 0.2 1"/>
        <light pos="2 2 3" dir="-1 -1 -1"/>
        
        <!-- 使用金属材质 -->
        <body name="metal_ball" pos="-0.8 0 0.5">
            <freejoint/>
            <geom type="sphere" size="0.2" material="metal"/>
        </body>
        
        <!-- 使用塑料材质 -->
        <body name="plastic_box" pos="0 0 0.5">
            <freejoint/>
            <geom type="box" size="0.15 0.15 0.15" material="plastic"/>
        </body>
        
        <!-- 使用玻璃材质（半透明） -->
        <body name="glass_cylinder" pos="0.8 0 0.5">
            <freejoint/>
            <geom type="cylinder" size="0.12 0.25" material="glass"/>
        </body>
        
        <!-- 直接使用RGBA -->
        <body name="colored_capsule" pos="0 -0.8 0.5">
            <freejoint/>
            <geom type="capsule" size="0.1 0.2" rgba="1 0 1 1"/>
        </body>
    </worldbody>
</mujoco>
"""

# 保存并加载
xml_path_materials = os.path.join(tutorial_dir, '04_materials.xml')
with open(xml_path_materials, 'w') as f:
    f.write(xml_content_materials)

model_mat = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_materials))
data_mat = mujoco.MjData(model_mat)

# 渲染
renderer_mat = mujoco.Renderer(model_mat, height=480, width=640)
renderer_mat.update_scene(data_mat)
image_mat = renderer_mat.render()

plt.figure(figsize=(12, 8))
plt.imshow(image_mat)
plt.title('不同材质效果对比', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ 材质演示场景创建成功！")
print("\n🎨 材质属性说明:")
print("   specular: 镜面反射强度 (0-1)")
print("   shininess: 光泽度 (0-1)")
print("   reflectance: 反射率 (0-1)")
print("   rgba: 颜色和透明度 (R G B A, 0-1)")

### 🎨 颜色说明

RGBA格式：`rgba="R G B A"`
- **R** (Red): 红色通道，0-1
- **G** (Green): 绿色通道，0-1  
- **B** (Blue): 蓝色通道，0-1
- **A** (Alpha): 透明度，0=完全透明，1=完全不透明

常用颜色示例：
```xml
红色: rgba="1 0 0 1"
绿色: rgba="0 1 0 1"
蓝色: rgba="0 0 1 1"
黄色: rgba="1 1 0 1"
白色: rgba="1 1 1 1"
黑色: rgba="0 0 0 1"
灰色: rgba="0.5 0.5 0.5 1"
半透明蓝: rgba="0 0 1 0.5"
```

---

## 6. 物理属性
## 6. Physical Properties

真实的物理仿真需要设置物体的物理属性：
- **质量 (mass/density)**: 影响惯性
- **摩擦力 (friction)**: 影响滑动和抓取
- **弹性 (solref/solimp)**: 影响碰撞反弹
- **阻尼 (damping)**: 影响运动衰减

In [ ]:
# ⚙️ 示例4: 物理属性对比
# Example 4: Physical properties comparison

xml_content_physics = """
<mujoco model="physics_demo">
    <option gravity="0 0 -9.81" timestep="0.002"/>
    
    <worldbody>
        <!-- 地面 -->
        <geom name="ground" type="plane" size="3 3 0.1" 
              friction="0.8 0.1 0.1" rgba="0.8 0.8 0.8 1"/>
        <light pos="0 0 3" dir="0 0 -1"/>
        
        <!-- 重球 (密度高) -->
        <body name="heavy_ball" pos="-1 0 2">
            <freejoint/>
            <geom type="sphere" size="0.1" density="2000" 
                  friction="0.8 0.1 0.1" rgba="0.8 0.2 0.2 1"/>
        </body>
        
        <!-- 轻球 (密度低) -->
        <body name="light_ball" pos="-0.3 0 2">
            <freejoint/>
            <geom type="sphere" size="0.1" density="100" 
                  friction="0.8 0.1 0.1" rgba="0.2 0.8 0.2 1"/>
        </body>
        
        <!-- 高弹性球 (弹力大) -->
        <body name="bouncy_ball" pos="0.4 0 2">
            <freejoint/>
            <geom type="sphere" size="0.1" density="1000" 
                  solref="0.002 1" solimp="0.99 0.99 0.001" 
                  friction="0.8 0.1 0.1" rgba="0.2 0.2 0.8 1"/>
        </body>
        
        <!-- 低摩擦球 (滑溜) -->
        <body name="slippery_ball" pos="1.1 0 2">
            <freejoint/>
            <geom type="sphere" size="0.1" density="1000" 
                  friction="0.1 0.005 0.0001" rgba="0.8 0.8 0.2 1"/>
        </body>
    </worldbody>
</mujoco>
"""

# 保存并加载
xml_path_physics = os.path.join(tutorial_dir, '05_physics.xml')
with open(xml_path_physics, 'w') as f:
    f.write(xml_content_physics)

model_phys = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_physics))
data_phys = mujoco.MjData(model_phys)

print("✅ 物理属性演示场景创建成功！")
print("\n⚙️ 测试对象:")
print("   🔴 重球: 高密度 (2000 kg/m³)")
print("   🟢 轻球: 低密度 (100 kg/m³)")
print("   🔵 弹球: 高弹性系数")
print("   🟡 滑球: 低摩擦系数")

In [ ]:
# 🎬 对比物理仿真效果
# Compare physics simulation effects

renderer_phys = mujoco.Renderer(model_phys, height=480, width=640)

# 记录不同时刻的图像
timestamps = [0, 50, 100, 150]
images_phys = []

for i in range(200):
    mujoco.mj_step(model_phys, data_phys)
    
    if i in timestamps:
        renderer_phys.update_scene(data_phys)
        images_phys.append(renderer_phys.render())

# 显示时间序列
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, (img, t) in enumerate(zip(images_phys, timestamps)):
    axes[idx].imshow(img)
    axes[idx].set_title(f'时刻 t={t} steps', fontsize=12)
    axes[idx].axis('off')

plt.suptitle('物理属性对比：观察不同球体的运动差异', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("✅ 物理仿真完成！")
print("\n观察结果:")
print("   - 重球下落最快，落地后弹跳最小")
print("   - 轻球下落较慢，受空气阻力影响大")
print("   - 弹球会多次弹跳")
print("   - 滑球落地后会滚动很远")

### ⚙️ 物理参数说明

#### 1. 质量和密度
```xml
<!-- 方法1: 直接指定质量 -->
<geom mass="1.5"/>

<!-- 方法2: 指定密度（推荐） -->
<geom density="1000"/>  <!-- kg/m³ -->
```

#### 2. 摩擦力
```xml
<!-- friction="sliding torsion rolling" -->
<geom friction="0.8 0.1 0.1"/>
```
- **sliding**: 滑动摩擦系数 (最重要)
- **torsion**: 扭转摩擦系数
- **rolling**: 滚动摩擦系数

#### 3. 碰撞参数
```xml
<!-- 控制碰撞的软硬程度和弹性 -->
<geom solref="0.002 1" solimp="0.9 0.95 0.001"/>
```
- **solref**: 刚度和阻尼
- **solimp**: 阻抗参数

---

## 7. 摄像头和光源
## 7. Cameras and Lights

摄像头和光源对于观察和渲染场景至关重要。

In [ ]:
# 📷 示例5: 多摄像头视角
# Example 5: Multiple camera views

xml_content_cameras = """
<mujoco model="camera_demo">
    <worldbody>
        <geom name="ground" type="plane" size="2 2 0.1" rgba="0.8 0.8 0.8 1"/>
        
        <!-- 光源 -->
        <light name="top_light" pos="0 0 3" dir="0 0 -1" diffuse="0.8 0.8 0.8"/>
        <light name="side_light" pos="2 2 1" dir="-1 -1 -0.5" diffuse="0.5 0.5 0.5"/>
        
        <!-- 中心物体 -->
        <body name="center_obj" pos="0 0 0.5">
            <freejoint/>
            <geom type="box" size="0.2 0.2 0.2" rgba="1 0.5 0 1"/>
        </body>
        
        <!-- 摄像头1: 俯视图 -->
        <camera name="topview" pos="0 0 2.5" quat="1 0 0 0" fovy="50"/>
        
        <!-- 摄像头2: 正面视图 -->
        <camera name="frontview" pos="2 0 1" quat="0.707 0.707 0 0" fovy="50"/>
        
        <!-- 摄像头3: 侧面视图 -->
        <camera name="sideview" pos="0 2 1" quat="0.707 0 0 0.707" fovy="50"/>
        
        <!-- 摄像头4: 斜视图 -->
        <camera name="perspectiveview" pos="1.5 1.5 1.5" xyaxes="-1 1 0 -0.577 -0.577 1.154" fovy="50"/>
    </worldbody>
</mujoco>
"""

# 保存并加载
xml_path_cameras = os.path.join(tutorial_dir, '06_cameras.xml')
with open(xml_path_cameras, 'w') as f:
    f.write(xml_content_cameras)

model_cam = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_cameras))
data_cam = mujoco.MjData(model_cam)

print(f"✅ 多摄像头场景创建成功！")
print(f"   摄像头数量: {model_cam.ncam}")
print(f"   光源数量: {model_cam.nlight}")

In [ ]:
# 🎥 渲染所有摄像头视角
# Render all camera views

renderer_cam = mujoco.Renderer(model_cam, height=400, width=400)

camera_names = ['topview', 'frontview', 'sideview', 'perspectiveview']
camera_images = []

for cam_name in camera_names:
    renderer_cam.update_scene(data_cam, camera=cam_name)
    img = renderer_cam.render()
    camera_images.append(img)

# 显示所有视角
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

titles = ['俯视图 (Top View)', '正面视图 (Front View)', 
          '侧面视图 (Side View)', '透视图 (Perspective View)']

for idx, (img, title) in enumerate(zip(camera_images, titles)):
    axes[idx].imshow(img)
    axes[idx].set_title(title, fontsize=12)
    axes[idx].axis('off')

plt.suptitle('多摄像头视角对比', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("✅ 所有摄像头视角渲染成功！")

### 📷 摄像头参数说明

#### 位置和方向
```xml
<!-- 方法1: 使用位置和四元数 -->
<camera pos="x y z" quat="w x y z" fovy="45"/>

<!-- 方法2: 使用位置和目标点 -->
<camera pos="x y z" target="x y z" fovy="45"/>

<!-- 方法3: 使用坐标轴 -->
<camera pos="x y z" xyaxes="x1 y1 z1 x2 y2 z2" fovy="45"/>
```

#### 视野角度 (Field of View)
- **fovy**: 垂直方向视野角度（度数）
- 典型值：30-90度
- 更大的fovy = 更广的视野（鱼眼效果）

### 💡 光源参数

```xml
<light pos="x y z" dir="dx dy dz" 
       diffuse="r g b" specular="r g b"/>
```
- **pos**: 光源位置
- **dir**: 光照方向
- **diffuse**: 漫反射颜色
- **specular**: 镜面反射颜色

---

## 8. 可视化和渲染
## 8. Visualization and Rendering

MuJoCo提供多种渲染选项来可视化仿真场景。

In [ ]:
# 🎨 示例6: 高级渲染选项
# Example 6: Advanced rendering options

# 创建一个复杂场景
xml_content_render = """
<mujoco model="render_demo">
    <worldbody>
        <geom name="ground" type="plane" size="3 3 0.1" rgba="0.5 0.5 0.5 1"/>
        <light pos="0 0 3" dir="0 0 -1"/>
        
        <!-- 创建一个金字塔结构 -->
        <!-- 第一层 -->
        <body name="b1" pos="-0.3 -0.3 0.1">
            <geom type="box" size="0.1 0.1 0.1" rgba="1 0 0 1"/>
        </body>
        <body name="b2" pos="0.3 -0.3 0.1">
            <geom type="box" size="0.1 0.1 0.1" rgba="0 1 0 1"/>
        </body>
        <body name="b3" pos="-0.3 0.3 0.1">
            <geom type="box" size="0.1 0.1 0.1" rgba="0 0 1 1"/>
        </body>
        <body name="b4" pos="0.3 0.3 0.1">
            <geom type="box" size="0.1 0.1 0.1" rgba="1 1 0 1"/>
        </body>
        
        <!-- 第二层 -->
        <body name="b5" pos="0 0 0.3">
            <geom type="sphere" size="0.15" rgba="1 0 1 1"/>
        </body>
        
        <camera name="view" pos="1.5 1.5 1" xyaxes="-1 1 0 -0.5 -0.5 1" fovy="50"/>
    </worldbody>
</mujoco>
"""

xml_path_render = os.path.join(tutorial_dir, '07_render_demo.xml')
with open(xml_path_render, 'w') as f:
    f.write(xml_content_render)

model_render = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_render))
data_render = mujoco.MjData(model_render)

print("✅ 渲染演示场景创建成功！")

In [ ]:
# 🎨 不同渲染模式对比
# Compare different rendering modes

renderer_demo = mujoco.Renderer(model_render, height=400, width=400)

# 设置不同的渲染标志
render_configs = [
    {'name': '正常渲染', 'flags': {}},
    {'name': '线框模式', 'flags': {'wireframe': True}},
    {'name': '显示坐标轴', 'flags': {'frame': True}},
    {'name': '显示接触点', 'flags': {'contact': True}},
]

render_images = []
for config in render_configs:
    # 重置渲染器场景标志
    renderer_demo.update_scene(data_render, camera='view')
    
    # 设置可视化标志（注意：这需要修改scene对象）
    if 'wireframe' in config['flags']:
        # 线框模式需要特殊处理
        pass
    
    img = renderer_demo.render()
    render_images.append((config['name'], img))

# 显示对比
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, (name, img) in enumerate(render_images):
    axes[idx].imshow(img)
    axes[idx].set_title(name, fontsize=12)
    axes[idx].axis('off')

plt.suptitle('不同渲染模式对比', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("✅ 渲染模式对比完成！")

In [ ]:
# 🎬 创建动画：旋转视角
# Create animation: rotating view

import math

print("🎬 生成旋转视角动画...")

animation_frames = []
num_frames = 36  # 360度 / 10度 = 36帧

for i in range(num_frames):
    angle = (i / num_frames) * 2 * math.pi
    radius = 2.0
    
    # 计算摄像头位置（圆周运动）
    cam_x = radius * math.cos(angle)
    cam_y = radius * math.sin(angle)
    cam_z = 1.0
    
    # 修改摄像头位置
    cam_id = mujoco.mj_name2id(model_render, mujoco.mjtObj.mjOBJ_CAMERA, 'view')
    model_render.cam_pos[cam_id] = [cam_x, cam_y, cam_z]
    
    # 渲染
    renderer_demo.update_scene(data_render, camera='view')
    img = renderer_demo.render()
    animation_frames.append(img)

# 显示部分关键帧
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

key_frames = [0, 6, 12, 18, 24, 30]
for idx, frame_idx in enumerate(key_frames):
    axes[idx].imshow(animation_frames[frame_idx])
    axes[idx].set_title(f'角度: {frame_idx*10}°', fontsize=10)
    axes[idx].axis('off')

plt.suptitle('旋转视角动画 - 关键帧', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print(f"✅ 动画生成完成！共 {num_frames} 帧")
print("   提示：可以使用imageio库将帧序列保存为GIF或MP4")

---

## 9. 保存自定义场景
## 9. Save Custom Scene

现在让我们综合所学知识，创建一个完整的自定义场景。

In [ ]:
# 🎯 综合练习：创建你的第一个完整场景
# Comprehensive exercise: Create your first complete scene

def create_custom_scene(scene_name, output_path):
    """
    创建一个自定义MuJoCo场景
    
    参数:
        scene_name: 场景名称
        output_path: 输出文件路径
    """
    
    xml_template = f"""
<mujoco model="{scene_name}">
    <!-- ==================== 全局设置 ==================== -->
    <option gravity="0 0 -9.81" timestep="0.002" integrator="RK4"/>
    
    <!-- ==================== 资源定义 ==================== -->
    <asset>
        <!-- 材质 -->
        <material name="floor_mat" rgba="0.4 0.4 0.4 1" 
                  specular="0.5" shininess="0.5"/>
        <material name="wood" rgba="0.8 0.6 0.4 1" 
                  specular="0.3" shininess="0.2"/>
        <material name="metal" rgba="0.7 0.7 0.8 1" 
                  specular="1" shininess="0.9"/>
    </asset>
    
    <!-- ==================== 世界物体 ==================== -->
    <worldbody>
        <!-- 地面 -->
        <geom name="floor" type="plane" size="5 5 0.1" 
              material="floor_mat" friction="0.8 0.1 0.1"/>
        
        <!-- 光源 -->
        <light name="main_light" pos="2 2 4" dir="-0.5 -0.5 -1" 
               diffuse="1 1 1" specular="0.5 0.5 0.5"/>
        <light name="fill_light" pos="-2 -2 3" dir="0.5 0.5 -1" 
               diffuse="0.5 0.5 0.5" specular="0.2 0.2 0.2"/>
        
        <!-- 桌子 -->
        <body name="table" pos="0 0 0">
            <!-- 桌面 -->
            <geom name="table_top" type="box" size="1 0.8 0.05" 
                  pos="0 0 0.6" material="wood"/>
            <!-- 桌腿 -->
            <geom name="leg1" type="cylinder" size="0.05 0.3" 
                  pos="0.8 0.6 0.3" material="wood"/>
            <geom name="leg2" type="cylinder" size="0.05 0.3" 
                  pos="-0.8 0.6 0.3" material="wood"/>
            <geom name="leg3" type="cylinder" size="0.05 0.3" 
                  pos="0.8 -0.6 0.3" material="wood"/>
            <geom name="leg4" type="cylinder" size="0.05 0.3" 
                  pos="-0.8 -0.6 0.3" material="wood"/>
        </body>
        
        <!-- 桌面上的物体 -->
        <body name="object1" pos="-0.3 0 0.75">
            <freejoint/>
            <geom type="sphere" size="0.08" density="1000" 
                  friction="0.8 0.1 0.1" rgba="1 0.2 0.2 1"/>
        </body>
        
        <body name="object2" pos="0.3 0 0.75">
            <freejoint/>
            <geom type="box" size="0.08 0.08 0.08" density="1000" 
                  friction="0.8 0.1 0.1" material="metal"/>
        </body>
        
        <!-- 摄像头 -->
        <camera name="front_view" pos="2 0 1.5" xyaxes="0 1 0 -0.5 0 1" fovy="45"/>
        <camera name="top_view" pos="0 0 3" quat="1 0 0 0" fovy="50"/>
        <camera name="side_view" pos="0 2.5 1.5" xyaxes="-1 0 0 0 -0.5 1" fovy="45"/>
    </worldbody>
</mujoco>
    """
    
    # 保存文件
    with open(output_path, 'w') as f:
        f.write(xml_template)
    
    print(f"✅ 场景文件创建成功: {output_path}")
    return output_path

# 创建自定义场景
custom_scene_path = os.path.join(tutorial_dir, '08_my_first_scene.xml')
create_custom_scene("my_first_scene", custom_scene_path)

print("\n📋 场景包含:")
print("   ✓ 带纹理的地面")
print("   ✓ 木质桌子（桌面+4条腿）")
print("   ✓ 桌面上的两个物体（球体和立方体）")
print("   ✓ 两个光源（主光源+补光）")
print("   ✓ 三个摄像头（正面、顶部、侧面）")

In [ ]:
# 🎬 加载并可视化自定义场景
# Load and visualize custom scene

model_custom_scene = mujoco.MjModel.from_xml_path(os.path.abspath(custom_scene_path))
data_custom_scene = mujoco.MjData(model_custom_scene)

print("✅ 自定义场景加载成功！")
print(f"\n📊 场景统计:")
print(f"   物体数量: {model_custom_scene.nbody}")
print(f"   几何体数量: {model_custom_scene.ngeom}")
print(f"   摄像头数量: {model_custom_scene.ncam}")
print(f"   光源数量: {model_custom_scene.nlight}")

In [ ]:
# 📸 渲染自定义场景的所有视角
# Render all views of custom scene

renderer_custom_scene = mujoco.Renderer(model_custom_scene, height=480, width=640)

views = ['front_view', 'top_view', 'side_view']
view_images = []

for view_name in views:
    renderer_custom_scene.update_scene(data_custom_scene, camera=view_name)
    img = renderer_custom_scene.render()
    view_images.append(img)

# 显示所有视角
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

titles = ['正面视角', '俯视视角', '侧面视角']
for idx, (img, title) in enumerate(zip(view_images, titles)):
    axes[idx].imshow(img)
    axes[idx].set_title(title, fontsize=14)
    axes[idx].axis('off')

plt.suptitle('我的第一个MuJoCo场景 - 多视角展示', fontsize=18, y=1.02)
plt.tight_layout()
plt.show()

print("🎉 恭喜！你已成功创建并渲染了第一个完整的MuJoCo场景！")

In [ ]:
# 🎬 运行物理仿真
# Run physics simulation

print("🎬 开始物理仿真...物体将落到桌面上\n")

# 运行300步仿真
for i in range(300):
    mujoco.mj_step(model_custom_scene, data_custom_scene)

# 渲染最终状态
renderer_custom_scene.update_scene(data_custom_scene, camera='front_view')
final_image = renderer_custom_scene.render()

# 对比初始和最终状态
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].imshow(view_images[0])  # 初始状态
axes[0].set_title('初始状态：物体悬浮', fontsize=14)
axes[0].axis('off')

axes[1].imshow(final_image)  # 最终状态
axes[1].set_title('最终状态：物体落在桌面上', fontsize=14)
axes[1].axis('off')

plt.suptitle('物理仿真前后对比', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("✅ 物理仿真完成！")
print("   观察：物体在重力作用下落到桌面，遵循真实物理定律")

---

## 🎓 教程总结
## Tutorial Summary

### ✅ 你已经学会了：

1. **MuJoCo基础概念**
   - XML配置文件结构
   - Model和Data对象
   - 物理仿真循环

2. **几何形状**
   - 7种基本几何类型
   - 尺寸参数设置
   - 位置和方向控制

3. **材质和外观**
   - RGBA颜色
   - 材质属性（反光、光泽）
   - 透明度控制

4. **物理属性**
   - 质量和密度
   - 摩擦系数
   - 碰撞参数

5. **摄像头和光源**
   - 多摄像头配置
   - 视角控制
   - 光照设置

6. **可视化技术**
   - 图像渲染
   - 多视角显示
   - 动画生成

### 📁 本教程生成的文件

所有场景文件保存在 `./tutorial_scenes/` 目录：
- `01_simple_scene.xml` - 最简单的场景
- `02_shapes_gallery.xml` - 几何形状画廊
- `03_my_shape.xml` - 你的自定义形状
- `04_materials.xml` - 材质演示
- `05_physics.xml` - 物理属性对比
- `06_cameras.xml` - 多摄像头示例
- `07_render_demo.xml` - 渲染演示
- `08_my_first_scene.xml` - 你的第一个完整场景

### 🚀 下一步学习

继续学习 **Part 2 - 机器人和夹爪** (`11.mujoco_tutorial_robot.ipynb`)，你将学会：
- 添加机器人模型
- 关节和执行器
- 夹爪控制
- 物体抓取仿真
- 传感器和控制器

---

## 💡 练习建议

在继续下一个教程前，尝试以下练习：

1. **简单练习**：创建一个包含5个不同形状和颜色的物体的场景
2. **中级练习**：设计一个多层塔结构（金字塔或楼梯）
3. **高级练习**：创建一个完整的房间场景（墙壁、地板、家具）

提示：复制并修改 `08_my_first_scene.xml` 作为起点！

---

## 📚 参考资源

- [MuJoCo官方文档](https://mujoco.readthedocs.io/)
- [XML参考手册](https://mujoco.readthedocs.io/en/stable/XMLreference.html)
- [Python绑定文档](https://mujoco.readthedocs.io/en/stable/python.html)

---

🎉 **恭喜完成MuJoCo入门教程Part 1！**